In [6]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 GPT API 키 가져오기
google_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
genai.configure(api_key=google_key)
model = genai.GenerativeModel('gemini-1.5-flash') 

In [ ]:
# 구글 api 키 확인
google_key

In [3]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
import json
import os

path = "chat_history_gemini.json"


# 대화 기록 로드 함수
def load_history():
    if os.path.exists(path):
        with open(path, "r", encoding = "utf-8") as file:
            return json.load(file)
    return [{"role" : "system", "content" : "당신은 친절한 화장품 안내 가이드입니다"}]

# 대화 기록 저장 함수
def save_history(messages):
    with open(path, "w", encoding = "utf-8",) as file:
        json.dump(messages, file, ensure_ascii = False, indent = 4)


# gpt 모델 호출 함수
def get_completion_from_messages(messages, temperature = 1):
    response = client.chat.completions.create(
        model = "gemini-1.5-flash",
        messages = messages,
        temperature = temperature
    )
    return response.choices[0].message.content

In [ ]:
import google.generativeai as genai
import json

# Google Gemini API 설정
genai.configure(api_key=google_key)

# 시스템 메시지 설정
system_instruction = """당신은 친절한 화장품 안내 가이드 챗봇명 피카추야 
질문자가 원하는 종류의 화장품을 추천해줘."""

model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# JSON 파일 경로
history_file = "chat_history_gemini.json"

# 대화 기록 로드 함수
def load_history():
    if os.path.exists(history_file): 
        with open(history_file, "r", encoding="utf-8") as file:
            return json.load(file)
    # 파일이 없으면 기본 시스템 메시지 반환
    return [{"role": "system", "parts": [system_instruction]}]

# 대화 기록 저장 함수
def save_history(messages):
    with open(history_file, "w", encoding="utf-8") as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

# 대화 시작
messages = []
print("챗봇(피카추) : 반갑습니다 화장품 안내 가이드 피카추입니다 !") 
while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)  # 대화 기록 저장
        break

    # 사용자 입력 추가
    messages.append({"role": "user", "parts": [user_input]})

    # AI 응답 생성
    response = model.generate_content(messages)

    # 응답 추가
    messages.append({"role": "assistant", "parts": [response.text]})

    # 대화 출력
    print(f"사용자 : {user_input} \n챗봇 : {response.text}\n")